In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
# from xgboost import XGBClassifier
RANDOM_STATE = 42

First I imported the data and investigated it. I found that I could merge my target and variables using the id column.

In [2]:
df = pd.read_csv('../Data/Training_set_values.csv')
target = pd.read_csv('../Data/Training_set_target.csv')

In [4]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
target.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [5]:
df, X_hold, target, y_hold = train_test_split(df, target, test_size=.1,
                                                    random_state=RANDOM_STATE) 

In [6]:
cleaning_df = df.merge(target)

In [7]:
cleaning_df.status_group.value_counts()

functional                 29062
non functional             20506
functional needs repair     3892
Name: status_group, dtype: int64

In [8]:
cleaning_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,31080,0.0,2012-10-10,Dwe,0,WEDECO,33.179596,-4.214966,Hospitali,0,...,coloured,colored,insufficient,insufficient,dam,dam,surface,dam,dam,functional
1,17282,50.0,2013-02-16,Jaica,-33,Tasaf,40.322805,-10.456297,Kwa Bakari Ismail,0,...,salty,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
2,72545,0.0,2011-03-20,Dhv,298,DWE,36.353508,-8.780099,Kwa Mgirigiti,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,other,other,non functional
3,44490,0.0,2012-10-12,Rwssp,0,RWSSP,33.071249,-3.803178,Ujamaa,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,non functional
4,67816,5.0,2013-02-04,Ces (gmbh),1088,DWE,37.271938,-3.283501,Kwa Richard Simon Chami,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional


I looked through all of the value counts for every column to find out which columns have data I am going to want to use. For instance: medium variety, low null value count, checking for null replacement values etc.

In [9]:
results = []
for column in cleaning_df.columns.to_list():
    results.append({column : cleaning_df[column].value_counts()})
results

[{'id': 0        1
  11951    1
  65013    1
  58870    1
  60919    1
          ..
  27352    1
  25305    1
  31450    1
  29403    1
  2047     1
  Name: id, Length: 53460, dtype: int64},
 {'amount_tsh': 0.0         37474
  500.0        2791
  50.0         2239
  20.0         1344
  1000.0       1311
              ...  
  5400.0          1
  200000.0        1
  53.0            1
  26.0            1
  59.0            1
  Name: amount_tsh, Length: 96, dtype: int64},
 {'date_recorded': 2011-03-17    511
  2013-02-03    501
  2011-03-15    500
  2011-03-14    480
  2011-03-16    446
               ... 
  2011-09-15      1
  2002-10-14      1
  2011-09-19      1
  2011-09-12      1
  2011-08-30      1
  Name: date_recorded, Length: 354, dtype: int64},
 {'funder': Government Of Tanzania    8156
  Danida                    2799
  Hesawa                    1999
  Rwssp                     1229
  World Bank                1208
                            ... 
  Serian                       1

Using the info I found digging through these results I found which columns needed to be dropped altogether and which needed some cleaning. So I dropped the columns, replaced some null value replacement values with actual null values and dropped them all. Then I replaced the rest of the replacement values with the mean. 

In [5]:
to_drop = ['scheme_name', 'amount_tsh', 'num_private', 'recorded_by', 'extraction_type', 'payment',
           'quality_group','quantity_group', 'region', 'waterpoint_type_group']
cleaning_df.drop(to_drop, axis=1, inplace=True)
cleaning_df['longitude'].replace({0 : np.NaN}, inplace=True)
cleaning_df['wpt_name'].replace({'none' : np.NaN}, inplace=True)
cleaning_df['scheme_management'].replace({'None' : np.NaN}, inplace=True)

cleaning_df.dropna(inplace=True)

In [6]:
replace_columns = ['construction_year', 'population']
for column in replace_columns:
    this_mean = round(cleaning_df[column][cleaning_df[column] != 0].mean())
    cleaning_df[column].replace({0 : this_mean}, inplace=True)

I next changed my target to intergers in order to run a classification model against the data. I also combined 'functional needs repair' and 'functional' so it is binary.

In [7]:
needs_repair_ids = cleaning_df['id'][cleaning_df['status_group'] == 'functional needs repair']
cleaning_df['status_group'].replace({'functional needs repair': 'functional'}, inplace=True)
target = cleaning_df['status_group']
preprocessed_df = cleaning_df.drop('status_group', axis=1)

In [9]:
target.replace({'functional' : 1 , 'non functional' : 0}, inplace=True)

I split my variables into categorical and continuous so I am able to then One Hot Encode my categorical data and concat my continuous back onto for my processed_df. I then save all my individual datasets in order to access them in other notebooks.

In [7]:
categorical = ['basin', 'public_meeting', 'scheme_management', 'permit',
               'extraction_type_group', 'extraction_type_class', 'management',
               'management_group', 'payment_type', 'water_quality', 'quantity',
               'source_type', 'source_class', 'waterpoint_type']
continuous = ['id', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population', 'construction_year']

In [11]:
categorical_df = preprocessed_df[categorical]
continuous_df = preprocessed_df[continuous]
dummy_df = pd.get_dummies(categorical_df)
processed_df = pd.concat([continuous_df, dummy_df], axis=1)

In [12]:
# processed_df.to_csv('../Data/processed_varibles.csv', index=False)
# target.to_csv('../Data/processed_target.csv', index=False)
# needs_repair_ids.to_csv('../Data/needs_repair_ids.csv', index=False)
# X_hold.to_csv('../Data/varibles_hold.csv', index=False)
# y_hold.to_csv('../Data/target_hold.csv', index=False)